# Build 3 region model - AB
https://github.com/Phyrise/nnUNet_translation/blob/master/notebooks/nnUNetv2_translation_tutorial_dualmod.ipynb

In [1]:
import os, glob, shutil, json
from pathlib import Path
import SimpleITK as sitk
import numpy as np
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm


## config

In [2]:
config = {
    "dataset_id": 300,  # Updated to 200 for CT noNorm
    "dataset_data_name": "synthrad2025_task1_MR_AB",
    "dataset_target_name": "synthrad2025_task1_CT_AB",
    "data_root": "/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Train/Task1/AB", # include centreD
    "preprocessing_CT": "CT", 
    "preprocessing_MRI": "MR",
    "preprocessing_mask": "no mask",
}

In [3]:
# save config to a JSON file
config_path = Path(f"config_{config['dataset_id']}.json")
with open(config_path, 'w') as f:
    json.dump(config, f, indent=4)

In [20]:

data_root = config["data_root"]
os.environ["nnUNet_raw"] = "/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/raw"
os.environ["nnUNet_preprocessed"] = "/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/preprocessed"
os.environ["nnUNet_results"] = "/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/results"

# example with 2 input modalities
list_data_mri = sorted(glob.glob(os.path.join(data_root, '**','mr.mha'), recursive=True))
# list_data_mask = sorted(glob.glob(os.path.join(data_root, '**','mask.mha'), recursive=True))
list_data_ct = sorted(glob.glob(os.path.join(data_root, '**','ct.mha'), recursive=True))


print("input1 ---", len(list_data_mri), list_data_mri)
# print("input2 ---", len(list_data_mask), list_data_mask)
print("target ---", len(list_data_ct), list_data_ct)

input1 --- 175 ['/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Train/Task1/AB/1ABA005/mr.mha', '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Train/Task1/AB/1ABA009/mr.mha', '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Train/Task1/AB/1ABA011/mr.mha', '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Train/Task1/AB/1ABA012/mr.mha', '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Train/Task1/AB/1ABA014/mr.mha', '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Train/Task1/AB/1ABA018/mr.mha', '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Train/Task1/AB/1ABA019/mr.mha', '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Train/Task1/AB/1ABA025/mr.mha', '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Train/Task1/AB/1A

## Define dataset ID and make paths

In [21]:
dataset_id = config["dataset_id"]
dataset_data_name = config["dataset_data_name"]
dataset_target_name = config["dataset_target_name"]

# we will copy the datas
# do not use exist_ok=True, we want an error if the dataset exist already
dataset_data_path = os.path.join(os.environ['nnUNet_raw'], f'Dataset{dataset_id:03d}_{dataset_data_name}') 
os.makedirs(dataset_data_path, exist_ok = True)
os.makedirs(os.path.join(dataset_data_path, 'imagesTr'), exist_ok=True)
os.makedirs(os.path.join(dataset_data_path, 'labelsTr'), exist_ok = True)

dataset_target_path = os.path.join(os.environ['nnUNet_raw'], f'Dataset{dataset_id+1:03d}_{dataset_target_name}') 
os.makedirs(dataset_target_path, exist_ok = True)
os.makedirs(os.path.join(dataset_target_path, 'imagesTr'), exist_ok = True)
os.makedirs(os.path.join(dataset_target_path, 'labelsTr'), exist_ok = True)

## Copy files and create dummy masks

In [22]:


def process_file(data_path, dataset_path, modality_suffix="_0000"):
    curr_img = sitk.ReadImage(data_path)
    filename = data_path.split(os.sep)[-2]
    if not filename.endswith(f'{modality_suffix}.mha'):
        filename = filename + f'{modality_suffix}.mha'
    sitk.WriteImage(curr_img, os.path.join(dataset_path, f'imagesTr/{filename}'))

    data = sitk.GetArrayFromImage(curr_img)
    data = np.ones_like(data)

    filename = filename.replace(modality_suffix, '')  # Remove modality suffix for masks
    label_path = os.path.join(dataset_path, f'labelsTr/{filename}')
    if not os.path.exists(label_path):
        label_img = sitk.GetImageFromArray(data)
        label_img.SetDirection(curr_img.GetDirection())
        label_img.SetOrigin(curr_img.GetOrigin())
        label_img.SetSpacing(curr_img.GetSpacing())
        sitk.WriteImage(label_img, label_path)

# Use the affine from the last MRI as a placeholder, but for sitk we use spacing/origin/direction from the image itself

with ThreadPoolExecutor() as executor:
    list(tqdm(executor.map(lambda data_path: process_file(data_path, dataset_data_path, "_0000"), list_data_mri), total=len(list_data_mri)))

# with ThreadPoolExecutor() as executor:
#     list(tqdm(executor.map(lambda data_path: process_file(data_path, dataset_data_path, "_0001"), list_data_mask), total=len(list_data_mask)))

with ThreadPoolExecutor() as executor:
    list(tqdm(executor.map(lambda target_path: process_file(target_path, dataset_target_path), list_data_ct), total=len(list_data_ct)))


  0%|          | 0/175 [00:00<?, ?it/s]

100%|██████████| 175/175 [01:13<00:00,  2.38it/s]


## Create dataset.json

In [23]:
# /!\ you will need to edit this with regards to the number of modalities used;
data_dataset_json = {
    "labels": {
        "label_001": "1", 
        "background": 0
    },
    "channel_names": {
        "0": config["preprocessing_MRI"],
        # "1": config["preprocessing_mask"],
        
    },
    "numTraining": len(list_data_mri),
    "file_ending": ".mha"
}
dump_data_datasets_path = os.path.join(dataset_data_path, 'dataset.json')
with open(dump_data_datasets_path, 'w') as f:
    json.dump(data_dataset_json, f)

target_dataset_json = {
    "labels": {
        "label_001": "1",
        "background": 0
    },
    "channel_names": {
        "0": config["preprocessing_CT"],
    },
    "numTraining": len(list_data_ct),
    "file_ending": ".mha"
}
dump_target_datasets_path = os.path.join(dataset_target_path, 'dataset.json')
with open(dump_target_datasets_path, 'w') as f:
    json.dump(target_dataset_json, f)

## Apply preprocessing and unpacking

In [24]:
if 'MPLBACKEND' in os.environ: 
    del os.environ['MPLBACKEND'] # avoid conflicts with matplotlib backend  
    
os.system(f'nnUNetv2_plan_and_preprocess -d {dataset_id} -c 3d_fullres')
os.system(f'nnUNetv2_unpack {dataset_id} 3d_fullres 0')

os.system(f'nnUNetv2_plan_and_preprocess -d {dataset_id + 1} -c 3d_fullres')
os.system(f'nnUNetv2_unpack {dataset_id + 1} 3d_fullres 0')

Fingerprint extraction...
Dataset300_synthrad2025_task1_MR_AB
Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> as reader/writer


100%|████████████████████████████████████████| 175/175 [00:58<00:00,  3.01it/s]


Experiment planning...
Using ZScoreNormalization for image normalization
Using ZScoreNormalization for image normalization
Attempting to find 3d_lowres config. 
Current spacing: [3.   1.03 1.03]. 
Current patch size: (np.int64(48), np.int64(192), np.int64(224)). 
Current median shape: [ 99.         429.12621359 451.45631068]
Using ZScoreNormalization for image normalization
Attempting to find 3d_lowres config. 
Current spacing: [3.     1.0609 1.0609]. 
Current patch size: (np.int64(48), np.int64(192), np.int64(224)). 
Current median shape: [ 99.         416.62739184 438.30709775]
Using ZScoreNormalization for image normalization
Attempting to find 3d_lowres config. 
Current spacing: [3.       1.092727 1.092727]. 
Current patch size: (np.int64(48), np.int64(192), np.int64(224)). 
Current median shape: [ 99.         404.49261343 425.5408716 ]
Using ZScoreNormalization for image normalization
Attempting to find 3d_lowres config. 
Current spacing: [3.         1.12550881 1.12550881]. 
Curre

100%|████████████████████████████████████████| 175/175 [03:57<00:00,  1.36s/it]


Using device: cuda:0

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

Fingerprint extraction...
Dataset301_synthrad2025_task1_CT_AB
Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> as reader/writer


100%|████████████████████████████████████████| 175/175 [01:20<00:00,  2.17it/s]


Experiment planning...
Using CTNormalization for image normalization
Using CTNormalization for image normalization
Attempting to find 3d_lowres config. 
Current spacing: [3.   1.03 1.03]. 
Current patch size: (np.int64(48), np.int64(192), np.int64(224)). 
Current median shape: [ 99.         429.12621359 451.45631068]
Using CTNormalization for image normalization
Attempting to find 3d_lowres config. 
Current spacing: [3.     1.0609 1.0609]. 
Current patch size: (np.int64(48), np.int64(192), np.int64(224)). 
Current median shape: [ 99.         416.62739184 438.30709775]
Using CTNormalization for image normalization
Attempting to find 3d_lowres config. 
Current spacing: [3.       1.092727 1.092727]. 
Current patch size: (np.int64(48), np.int64(192), np.int64(224)). 
Current median shape: [ 99.         404.49261343 425.5408716 ]
Using CTNormalization for image normalization
Attempting to find 3d_lowres config. 
Current spacing: [3.         1.12550881 1.12550881]. 
Current patch size: (np.i

100%|████████████████████████████████████████| 175/175 [04:39<00:00,  1.60s/it]


Using device: cuda:0

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################



0

## Copy mask raw data to the preprocessed folder


In [25]:
import shutil
def process_mask_file(data_path, dataset_mask_path):

    filename = data_path.split(os.sep)[-2]
    if not filename.endswith(f'.mha'):
        filename = filename + f'.mha'
    shutil.copy(data_path, os.path.join(dataset_mask_path, filename))


# Use the affine from the last MRI as a placeholder, but for sitk we use spacing/origin/direction from the image itself
dataset_mask_path = os.path.join(os.environ['nnUNet_preprocessed'], f'Dataset{dataset_id:03d}_{dataset_data_name}', 'masks')
os.makedirs(dataset_mask_path, exist_ok=True) 
with ThreadPoolExecutor() as executor:
    list(tqdm(executor.map(lambda data_path: process_mask_file(data_path, dataset_mask_path), list_data_mask), total=len(list_data_mask)))


100%|██████████| 175/175 [00:00<00:00, 209.43it/s]


## Define 2nd modality raw data as gt_seg of 1st modality

In [26]:
nnunet_datas_preprocessed_dir = os.path.join(os.environ['nnUNet_preprocessed'], f'Dataset{dataset_id+1:03d}_{dataset_target_name}') 
nnunet_targets_preprocessed_dir = os.path.join(os.environ['nnUNet_preprocessed'], f'Dataset{dataset_id:03d}_{dataset_data_name}') 

list_targets = glob.glob(os.path.join(f"{dataset_target_path}/imagesTr", '*'))
list_targets.sort()
list_gt_segmentations_datas = glob.glob(os.path.join(f"{nnunet_targets_preprocessed_dir}/gt_segmentations", '*'))
list_gt_segmentations_datas.sort()

print(nnunet_targets_preprocessed_dir)

for (preprocessed_path, gt_path) in zip(list_targets, list_gt_segmentations_datas):
    # here, gt_path is the path to the gt_segmentation in nnUNet_preprocessed.
    print(preprocessed_path, "->", gt_path) # ensure correct file pairing; 
    shutil.copy(src = preprocessed_path, dst = gt_path) # we use shutil.copy to ensure safety, but switching to shutil.move would be more efficient

/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/preprocessed/Dataset300_synthrad2025_task1_MR_AB
/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/raw/Dataset301_synthrad2025_task1_CT_AB/imagesTr/1ABA005_0000.mha -> /datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/preprocessed/Dataset300_synthrad2025_task1_MR_AB/gt_segmentations/1ABA005.mha
/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/raw/Dataset301_synthrad2025_task1_CT_AB/imagesTr/1ABA009_0000.mha -> /datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/preprocessed/Dataset300_synthrad2025_task1_MR_AB/gt_segmentations/1ABA009.mha
/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/raw/Dataset301_synthrad2025_task1_CT_AB/imagesTr/1ABA011_0000.mha -> /datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/preprocessed/Dataset3

## Define 2nd modality raw data as gt_seg of 1st modality

In [27]:
nnunet_datas_preprocessed_dir = os.path.join(os.environ['nnUNet_preprocessed'], f'Dataset{dataset_id+1:03d}_{dataset_target_name}') 
nnunet_targets_preprocessed_dir = os.path.join(os.environ['nnUNet_preprocessed'], f'Dataset{dataset_id:03d}_{dataset_data_name}') 

list_targets = glob.glob(os.path.join(f"{dataset_target_path}/imagesTr", '*'))
list_targets.sort()
list_gt_segmentations_datas = glob.glob(os.path.join(f"{nnunet_targets_preprocessed_dir}/gt_segmentations", '*'))
list_gt_segmentations_datas.sort()

print(nnunet_targets_preprocessed_dir)

for (preprocessed_path, gt_path) in zip(list_targets, list_gt_segmentations_datas):
    # here, gt_path is the path to the gt_segmentation in nnUNet_preprocessed.
    print(preprocessed_path, "->", gt_path) # ensure correct file pairing; 
    shutil.copy(src = preprocessed_path, dst = gt_path) # we use shutil.copy to ensure safety, but switching to shutil.move would be more efficient

/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/preprocessed/Dataset300_synthrad2025_task1_MR_AB
/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/raw/Dataset301_synthrad2025_task1_CT_AB/imagesTr/1ABA005_0000.mha -> /datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/preprocessed/Dataset300_synthrad2025_task1_MR_AB/gt_segmentations/1ABA005.mha
/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/raw/Dataset301_synthrad2025_task1_CT_AB/imagesTr/1ABA009_0000.mha -> /datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/preprocessed/Dataset300_synthrad2025_task1_MR_AB/gt_segmentations/1ABA009.mha
/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/raw/Dataset301_synthrad2025_task1_CT_AB/imagesTr/1ABA011_0000.mha -> /datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/preprocessed/Dataset3

## Define 2nd modality preprocessed files as ground truth of 1st modality


In [28]:
list_preprocessed_datas_seg_path = sorted(glob.glob(os.path.join(nnunet_targets_preprocessed_dir, 'nnUNetPlans_3d_fullres/*_seg.npy')))

list_preprocessed_targets_path = sorted(glob.glob(os.path.join(nnunet_datas_preprocessed_dir, 'nnUNetPlans_3d_fullres/*.npy')))
list_preprocessed_targets_path = [name for name in list_preprocessed_targets_path if '_seg' not in name]

for (datas_path, targets_path) in zip(list_preprocessed_datas_seg_path, list_preprocessed_targets_path):
    print(targets_path, "->", datas_path)
    shutil.copy(src = targets_path, dst = datas_path) 

/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/preprocessed/Dataset301_synthrad2025_task1_CT_AB/nnUNetPlans_3d_fullres/1ABA005.npy -> /datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/preprocessed/Dataset300_synthrad2025_task1_MR_AB/nnUNetPlans_3d_fullres/1ABA005_seg.npy
/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/preprocessed/Dataset301_synthrad2025_task1_CT_AB/nnUNetPlans_3d_fullres/1ABA009.npy -> /datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/preprocessed/Dataset300_synthrad2025_task1_MR_AB/nnUNetPlans_3d_fullres/1ABA009_seg.npy
/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/preprocessed/Dataset301_synthrad2025_task1_CT_AB/nnUNetPlans_3d_fullres/1ABA011.npy -> /datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/preprocessed/Dataset300_synthrad2025_task1_MR_AB/nnUNetPlans_3d_fullres/1ABA0

You should be able to start training with :
```
export nnUNet_raw="/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/raw"
export nnUNet_preprocessed="/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/preprocessed"
export nnUNet_results="/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/results"
```
nnUNetv2_train 200 3d_fullres 0 -tr nnUNetTrainerMRCT


In [29]:
os.system(f'nnUNetv2_train {dataset_id} 3d_fullres 0 -tr nnUNetTrainerMRCT')

Using device: cuda:0

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################


This is the configuration used by this training:
Configuration name: 3d_fullres
 {'data_identifier': 'nnUNetPlans_3d_fullres', 'preprocessor_name': 'DefaultPreprocessor', 'batch_size': 2, 'patch_size': [48, 192, 224], 'median_image_size_in_voxels': [99.0, 442.0, 465.0], 'spacing': [3.0, 1.0, 1.0], 'normalization_schemes': ['ZScoreNormalization'], 'use_mask_for_norm': [False], 'resampling_fn_data': 'resample_data_or_seg_to_shape', 'resampling_fn_seg': 'resample_data_or_seg_to_shape', 'resampling_fn_data_kwargs': {'is_seg': False, 'order': 3, 'order_z': 0, 'force_separat

Traceback (most recent call last):
  File "/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/envs/nnunet_trans/bin/nnUNetv2_train", line 8, in <module>
    sys.exit(run_training_entry())
  File "/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/ref/nnUNet_translation/nnunetv2/run/run_training.py", line 302, in run_training_entry
    run_training(args.dataset_name_or_id, args.configuration, args.fold, args.tr, args.p, args.pretrained_weights,
  File "/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/ref/nnUNet_translation/nnunetv2/run/run_training.py", line 238, in run_training
    nnunet_trainer.run_training()
  File "/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/ref/nnUNet_translation/nnunetv2/training/nnUNetTrainer/nnUNetTrainer.py", line 1296, in run_training
    train_outputs.append(self.train_step(next(self.dataloader_train)))
  File "/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/ref/nnUNet_translation/nnunetv2/tr

2